In [1]:
# take one line (=one board)
def printBoard(puzzle):
    a = 0
    for i in range(6):
        print(puzzle[a:(a+6)])
        a += 6
    print("\n")

In [2]:
class Car:
    def __init__(self, name, fuel, coordinate, orientation):
        
        self.name = name
        self.fuel = fuel
        self.coordinate = coordinate # list of coordinates that the car present in board
        self.orientation = orientation
    
    def carInfo(self):
        print("Name:",self.name , ", Fuel:", self.fuel,", coordinate:",self.coordinate,", orientation:",self.orientation)
        


In [3]:
class Board:

    def __init__(self, line):
        
        self.board = []
        self.cars = [] # list of car objects present in the board
        d = 6 # dimension
    
        carLetters =   ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

        # empty board
        myMatrix = [[0 for x in range(d)] for x in range(d)] 


        # making boards (36 characters)
        a = 0    
        for i in range(6):
            for j in range(6):
                myMatrix[i][j] = line[a]
                a = a+1
        
        # assign board to object
        self.board = myMatrix
        
        
        
        # coordinate of cars
        coordinateDict = {}
        for each in carLetters:
            if each in line: # if car exist in board
                coordinateDict[each] = [[x, y] for x, li in enumerate(self.board) for y, val in enumerate(li) if val==each]

                
        # fuel
        fuelDict = {}
        fuelInfoStr = line[36:].strip()
        if fuelInfoStr != "": # additional info exist
            fuels = fuelInfoStr.split()
            for each in fuels:
                fuelDict[each[0]] = int(each[1:])
            
            
        
        # make car object
        for carName in coordinateDict:
            
            # orientation
            if coordinateDict[carName][0][0] == coordinateDict[carName][1][0]:
                orientation = "horizontal"
            else:
                orientation = "vertical"
            
            if carName in fuelDict:
                self.cars.append(Car(carName, fuelDict[carName], coordinateDict[carName],orientation))
        
            else:
                self.cars.append(Car(carName, 100, coordinateDict[carName],orientation))

                                     

                        
                                     
                                


In [4]:
testLine = "BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII J0 B4"

In [5]:
hey = Board(testLine)

In [6]:
hey.board

[['B', 'B', 'B', '.', '.', 'M'],
 ['C', 'C', 'D', 'D', '.', 'M'],
 ['A', 'A', 'K', 'L', '.', 'M'],
 ['J', '.', 'K', 'L', 'E', 'E'],
 ['J', '.', 'G', 'G', '.', '.'],
 ['J', 'H', 'H', 'H', 'I', 'I']]

In [7]:
for each in hey.cars:
    each.carInfo()

Name: A , Fuel: 100 , coordinate: [[2, 0], [2, 1]] , orientation: horizontal
Name: B , Fuel: 4 , coordinate: [[0, 0], [0, 1], [0, 2]] , orientation: horizontal
Name: C , Fuel: 100 , coordinate: [[1, 0], [1, 1]] , orientation: horizontal
Name: D , Fuel: 100 , coordinate: [[1, 2], [1, 3]] , orientation: horizontal
Name: E , Fuel: 100 , coordinate: [[3, 4], [3, 5]] , orientation: horizontal
Name: G , Fuel: 100 , coordinate: [[4, 2], [4, 3]] , orientation: horizontal
Name: H , Fuel: 100 , coordinate: [[5, 1], [5, 2], [5, 3]] , orientation: horizontal
Name: I , Fuel: 100 , coordinate: [[5, 4], [5, 5]] , orientation: horizontal
Name: J , Fuel: 0 , coordinate: [[3, 0], [4, 0], [5, 0]] , orientation: vertical
Name: K , Fuel: 100 , coordinate: [[2, 2], [3, 2]] , orientation: vertical
Name: L , Fuel: 100 , coordinate: [[2, 3], [3, 3]] , orientation: vertical
Name: M , Fuel: 100 , coordinate: [[0, 5], [1, 5], [2, 5]] , orientation: vertical


In [8]:
file_path = "SampleInputOutput/Sample/sample-input.txt"
hashtag = "#"

def readPuzzles(file_path):
    with open(file_path) as file:
        puzzles = [line.rstrip() for line in file]
        puzzles = list(filter(None, puzzles))
        
        for line in puzzles.copy():
            if hashtag in line:
                puzzles.remove(line)
    return puzzles  


In [9]:
lines = readPuzzles(file_path)

In [10]:
for each in lines:
    test = Board(each)
    for car in test.cars:
        car.carInfo()
    print("\n")

Name: A , Fuel: 100 , coordinate: [[2, 3], [2, 4]] , orientation: horizontal
Name: B , Fuel: 100 , coordinate: [[0, 0], [0, 1]] , orientation: horizontal
Name: C , Fuel: 100 , coordinate: [[1, 4], [1, 5]] , orientation: horizontal
Name: D , Fuel: 100 , coordinate: [[3, 1], [3, 2]] , orientation: horizontal
Name: F , Fuel: 100 , coordinate: [[5, 2], [5, 3]] , orientation: horizontal
Name: G , Fuel: 100 , coordinate: [[3, 0], [4, 0], [5, 0]] , orientation: vertical
Name: H , Fuel: 100 , coordinate: [[4, 1], [5, 1]] , orientation: vertical
Name: I , Fuel: 100 , coordinate: [[0, 2], [1, 2], [2, 2]] , orientation: vertical
Name: J , Fuel: 100 , coordinate: [[0, 3], [1, 3]] , orientation: vertical
Name: K , Fuel: 100 , coordinate: [[3, 3], [4, 3]] , orientation: vertical
Name: L , Fuel: 100 , coordinate: [[4, 4], [5, 4]] , orientation: vertical
Name: M , Fuel: 100 , coordinate: [[2, 5], [3, 5]] , orientation: vertical


Name: A , Fuel: 100 , coordinate: [[2, 2], [2, 3]] , orientation: horizo